# OPERA RTC Validation: Terrain Flattening Part 2: Regression Analysis

**Rui Kawahara & Franz J Meyer; Alaska Satellite Facility, University of Alaska Fairbanks**

This notebook analyzes the terrain flattening performance of OPERA RTC products by analyzing the regression between the terrain flattened radar brightness and the local incidence angle information. 

#### _Objecive_: 

- Compare incidence angle maps for `GAMMA RTC` vs `OPERA RTC`.
- Plot the scatter plot of $Gamma_0$ vs Local Incidence Angle for:
    - `OPERA RTC`
    - `GAMMA RTC`
    
#### _Prerequisite_:

You will need the following:
- Local Iincidence Angle (`.tif`) and
- Backscatter (`.tif`) for:
    - `GAMMA RTC`
    - `OPERA RTC`
    
- For `OPERA RTC`, you will need to run following 2 notebooks:
    - `Prep_OPERA_RTC_CalVal_data_stage1_part1.ipynb`
    - `Prep_OPERA_RTC_CalVal_data_stage1_part2.ipynb`

#### _Important Notes_:

To avoid confusion, $Gamma_0$ will be referred to `backscatter` throughout this notebook.

(e.g.) `OPERA_L2_RTC-S1_S..._v0.3_VH_clip_masked_backscatter.tif`

<hr>

# 0. OPERA RTC Terrain Flattening Requirement

<div class="alert alert-success">
<i>The median radar backscatter of the OPERA RTC-S1 product over an area of foreslope shall be within 1dB of the median radar backscatter over an area of backslope for forested land-types, for at least 80% of all validation products considered.</i>
</div>

<hr>

# 1. Load Necessary Libraries

In [ ]:
from ipyfilechooser import FileChooser
import numpy as np
from pathlib import Path
from glob import glob
import shutil
from tqdm.auto import tqdm

import numpy.ma as ma
from osgeo import gdal, osr

import matplotlib.pyplot as plt
from PIL import Image

from scipy.stats import gaussian_kde

from matplotlib.colors import LinearSegmentedColormap
# "Viridis-like" colormap with white background
white_viridis = LinearSegmentedColormap.from_list('white_viridis', [
    (0, '#ffffff'),
    (1e-20, '#440053'),
    (0.2, '#404388'),
    (0.4, '#2a788e'),
    (0.6, '#21a784'),
    (0.8, '#78d151'),
    (1, '#fde624'),
], N=256)

<hr>

# 2. Get Path to `GAMMA RTC` and `OPERA RTC` Products

_NB_: Once selected, your path output should be highlighted in **green**

#### **GAMMA RTC**

Select the *`Tree_Cover` directory within `GAMMA RTC` dataset. At minimum, it should contain the following items:

- Local Incidence Angle (ends in following format: `*lc_inc_map_clip_deg.tif`)
- Two backscatter files (vv/vh files)

*_This name may differ depending on where you are getting the GAMMA RTC. It should contain the phrase `tree cover`._

In [ ]:
print("Select a file that contains the incidence angle map in your GAMMA RTC")
gamma_fc = FileChooser(Path.cwd())
display(gamma_fc)

#### **OPERA RTC**

Select the `Tree_Cover` directory within `OPERA RTC` dataset. At minimum, it should contain the following items:

- Local Incidence Angle (ends in following format: `*_local_incidence_angle_clip_deg.tif`)
- Two backscatter files (vv/vh files)

If you do not have this, generate the `OPERA RTC` using `Prep_OPERA_RTC_CalVal_data_stage1_part1&2`

In [ ]:
print("Select a file that contains the incidence angle map in your OPERA RTC")
opera_fc = FileChooser(Path.cwd())
display(opera_fc)

#### Define filepaths for incidence angle and backscatter for both RTCs

In [ ]:
gamma = Path(gamma_fc.selected_path)
opera = Path(opera_fc.selected_path)

# GAMMA RTC
gamma_vv = list(gamma.rglob("*VV_clip_masked_backscatter.tif"))[0]
gamma_vh = list(gamma.rglob("*VH_clip_masked_backscatter.tif"))[0]
gamma_inc_angle = list(gamma.rglob("*_lc_inc_map_clip_deg.tif"))[0]

# OPERA RTC
opera_vv = list(opera.rglob("*_VV_clip_masked_backscatter.tif"))[0]
opera_vh = list(opera.rglob("*_VH_clip_masked_backscatter.tif"))[0]
opera_inc_angle = list(opera.rglob("*_local_incidence_angle_clip_deg.tif"))[0]

<hr>

# 3. Compare Incidence Angle Maps of `GAMMA RTC` vs `OPERA RTC` Products

## 3.1 Define the Raveling Function for Comparing Incidence Angle Map

In [ ]:
"""
Dev notes:

No need to clean this.
Refactor to use the ravel part only
(We may not need this?)
"""

#fetch data cleans and ravels rasters. returns raveled raster
def fetch_data(raster_file, title="", null_value=-10):
    fname = raster_file
    # print(fname,0)
    
    #Using PIL to import raster, and convert it to numpy array
    im = Image.open(fname)
    j = np.array(im)
    
    #Data cleansing, remove the null value, in this case it's -10
    clean = lambda x: np.nan if x == null_value else x
    cleann = np.vectorize(clean)
    z = np.ravel(j) #ravelling to make a x or y axis
    y = cleann(z)
    retval = y[~np.isnan(y)]
    
    #some visualization to describe the data
    #plt.figure(figsize=(7,3))
    #plt.hist(retval, color='black', bins=32)
    #plt.title(title)
    
    #return raveled data
    return retval

## 3.2 Ravel the Data

In [ ]:
#raveling the data

print('This step may take a while...\n')

gamma_inc_angle_data = fetch_data(gamma_inc_angle, 'GAMMA RTC Local Incidence Angle Map')
opera_inc_angle_data = fetch_data(opera_inc_angle, 'OPERA RTC Local Incidence Angle Map')
opera_inc_angle_data = opera_inc_angle_data / (20.0 * np.pi)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(7,7))
ax[0].hist(gamma_inc_angle_data, color='black', bins=32)
ax[0].set_title('GAMMA RTC Local Incidence Angle Map')

ax[1].hist(opera_inc_angle_data, color='black', bins=32)
ax[1].set_title('OPERA RTC Local Incidence Angle Map')

<hr>

# 4. Generate Scatter Plot for `Backscatter` and `Local Incidence Angle` Maps

## 4.1 Helper Function for Plotting

Some helpful functions: to generate scatter plots and fit regression lines
- [`plt.scatter`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.scatter.html#matplotlib-pyplot-scatter)

In [ ]:
polarity = ['vv', 'vh']

def scatterplot(backscatter: float, inc_ang: float, title: str, x_axis_label: str, y_axis_label: str) -> None:
    """
    Given backscatter, incidence angle, and title name, it plots the scatter plot of {backscatter vs inc_ang}
    """
    
    a, b = np.polyfit(inc_ang, backscatter, 1)

    fig, ax = plt.subplots(figsize=(12,6))
    plt.rcParams.update({'font.size': 14})
    #ax.set_yticklabels([])
    #ax.set_xticklabels([])
    
    #Actually Plotting the data
    # x axis: backscatter
    # y axis: inc_ang
    #xy = np.vstack([inc_ang,backscatter])
    #z = gaussian_kde(xy)(xy)
    plt.hist2d(inc_ang,backscatter, bins=(300, 300), cmap=white_viridis)
   # plt.scatter(inc_ang,backscatter, c=z, s=100)
    
    #Making the graph pretty and informative!
    plt.title(f"{title} Scatter Plot", fontsize=28)
    plt.xlabel(f"{x_axis_label}", fontsize=22)
    plt.ylabel(f"{y_axis_label}", fontsize=22)
    plt.grid()
    plt.ylim([-25, 5])
    plt.plot(inc_ang, a*inc_ang+b, color='black', linestyle='-', linewidth=5)
    plt.text(5, 3, 'y = ' + '{:.4f}'.format(b) + ' + {:.4f}'.format(a) + 'x', size=18)
    output = f"{title}_regression.png"
    plt.savefig(output, dpi=300)
    
## Precondition Check - Confirm that the Backscatter and Incidence Angle are same size
def get_bs_incValue(backscatter, inc_ang) -> tuple:
    """
    Given path to backscatter and incidence angle (they must be in a PosixPath), 
    it will return a tuple containing a floating value of backscatter and incidence angle.
    
    It should return them in a following format:
    
    (value of backscatter, value of inc_ang)
    
    NB: Will throw an error if the shape/size of backscatter and inc_ang does not match
    """
    
    bs_open, inc_ang_open = gdal.Open(str(backscatter)), gdal.Open(str(inc_ang))
    bs_val, inc_ang_val = bs_open.ReadAsArray(), inc_ang_open.ReadAsArray()
    bs_val = 10.0 * np.log10(bs_val)
    mask = ~np.isnan(bs_val)
    bs_val = bs_val[mask]
    inc_ang_val = inc_ang_val[mask]
    bs_val = bs_val[::5]
    inc_ang_val = inc_ang_val[::5]
    if np.mean(inc_ang_val) > 1000.0:
        inc_ang_val = inc_ang_val / (20.0 * np.pi)
    
    # NOTE (May 3rd, 2023): != syntax will be deprecated in a future
    if bs_val.shape != inc_ang_val.shape:
        raise ValueError("Size of backscatter and incidence angle are different.")
    
    return (bs_val, inc_ang_val)

## 4.2 Plotting the Data (**GAMMA RTC**)

In [ ]:
for p in polarity:
    backscatter = gamma_vv if p == 'vv' else gamma_vh
    bs_inc_val = get_bs_incValue(backscatter, gamma_inc_angle)
    
    scatterplot(
            bs_inc_val[0], 
            bs_inc_val[1], 
            f'GAMMA RTC: {p} backscatter vs incident angle', 
            'x: Local Incidence Angle (Degrees)',
            f'y: Radar Brightness ({p})'
    )

## 4.3 Plotting the Data (**OPERA RTC**)

In [ ]:
for p in polarity:
    backscatter = opera_vv if p == 'vv' else opera_vh
    bs_inc_val = get_bs_incValue(backscatter, opera_inc_angle)
    
    scatterplot(
            bs_inc_val[0], 
            bs_inc_val[1], 
            f'OPERA RTC: {p} backscatter vs incident angle', 
            f'x: polarity ({p})',
            'y: incident angle'
    )

<hr>

In [ ]:

"""
Dev notes:

We need...

Two histograms (inc angles) (done)

Four scatterplot (vv/vh for GAMMA RTC inc angle and OPERA RTC's inc angle) (done)

Title the x-axis and y-axis appropriately (done)



Line fit (scatterplot -> poly fit) (not started)
https://www.python-graph-gallery.com/scatterplot-with-regression-fit-in-matplotlib

"""